In [31]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm
from statistics import mean, variance
from functools import lru_cache
from math import sqrt
from typing import List, Tuple
import itertools
import json
from IPython.display import clear_output
import os
import time

In [8]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [9]:
#Initializing parameters
input_shape = (28, 28, 1)
#batch_size = 64
#num_classes = 10
#epochs = 5

#Cleaning data
x_train=x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_train=x_train / 255.0
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_test=x_test/255.0

y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

In [10]:
def make_model(
    cnn_layer_sizes : List[Tuple[int, Tuple[int, int]]],
    dense_layer_sizes : List[int],
    activation_function : str
):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(input_shape))
    for i in range(len(cnn_layer_sizes)-1):
        model.add(tf.keras.layers.Conv2D(cnn_layer_sizes[i][0], cnn_layer_sizes[i][1], activation=activation_function))
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    if len(cnn_layer_sizes) > 0:
        model.add(tf.keras.layers.Conv2D(cnn_layer_sizes[-1][0], cnn_layer_sizes[-1][1], activation=activation_function))
    model.add(tf.keras.layers.Flatten())
    for elem in dense_layer_sizes:
        model.add(tf.keras.layers.Dense(elem, activation = activation_function))
    model.add(tf.keras.layers.Dense(10, activation = 'softmax'))
    model.compile(optimizer=tf.keras.optimizers.RMSprop(epsilon=1e-08), loss='categorical_crossentropy', metrics=['acc'])
    return model

def find_screens_that_mess_with_model(model):
    pass

In [11]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>0.995):
            print("\nReached 99.5% accuracy so cancelling training!")
            self.modelel.stop_training = True

"""callbacks = myCallback()

model = make_model([(10, (5,5)), (5, (3,3))], [64, 64], 'tanh')

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=5,
                    validation_split=0.1,
                    callbacks=[callbacks])"""

"callbacks = myCallback()\n\nmodel = make_model([(10, (5,5)), (5, (3,3))], [64, 64], 'tanh')\n\nhistory = model.fit(x_train, y_train,\n                    batch_size=batch_size,\n                    epochs=5,\n                    validation_split=0.1,\n                    callbacks=[callbacks])"

In [37]:
#CONFIGURATIONS
cnn_layer_depth = [1,2,3]
cnn_layer_dimensions = [(2,2), (3,3), (4,4), (5,5)]
num_cnn_layers = [1,2,3]
num_dense_layers = [1,2,3,4]
dense_layer_sizes = [16, 32, 64, 128, 256]
activation_functions = ['relu', 'sigmoid', 'tanh']
batch_sizes = [16, 32, 64, 128]

def gen_cnn_layer_combinations():
    L = []
    for layer in num_cnn_layers:
        for dim in cnn_layer_dimensions:
            L.append((layer, dim))
    return L

def send_to_github(filename):
    os.system(f"git add {filename}")
    os.system("git add model_info.json")
    os.system("git add stop_index.txt")
    os.system(f"git commit -m \"Added {filename}\"")
    os.system(f"git push origin main")
    os.system(f"rm -r {filename}")
    return
    
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>0.995):
            print("\nReached 99.5% accuracy so cancelling training!")
            self.modelel.stop_training = True

def iterate_combinations():
    try:
        with open('stop_index.txt', 'r') as file:
            stop_index = int(file.read())
            
    except:
        stop_index = -1
    
    counter = 0
    
    try:
        with open('model_info.json', 'r') as file:
            models_info = json.load(file)
    except FileNotFoundError:
        models_info = {
            "CNN" : [],
            "Dense" : [],
            "act_func" : [],
            "batch_size" : [],
            "filename" : [],
            "accuracy" : []
        }
    
    total = 1884 * 780 * 3 * 4
    start = time.time()
    for cnn_depth in cnn_layer_depth:
        for cnn_layer_dim in itertools.product(gen_cnn_layer_combinations(), repeat=cnn_depth):
            for dense_depth in num_dense_layers:
                for dense_layers in itertools.product(dense_layer_sizes, repeat=dense_depth):
                    for a_func in activation_functions:
                        for batch_size in batch_sizes:
                            print(f"{counter}/{total} - {round((counter / total) * 100, 2)}% - {round(time.time() - start, 2)}")
                            if counter > stop_index:
                                callbacks = myCallback()
                                model = make_model(list(cnn_layer_dim), list(dense_layers), a_func)
                                history = model.fit(x_train, y_train,
                                    batch_size=batch_size,
                                    epochs=2,
                                    validation_split=0.1,
                                    callbacks=[callbacks])
                                filename = f'model_configs/model{counter}_savefile'
                                model.save(filename)
                                send_to_github(filename)
                                models_info['CNN'].append(cnn_layer_dim)
                                models_info['Dense'].append(dense_layers)
                                models_info['act_func'].append(a_func)
                                models_info['batch_size'].append(batch_size)
                                models_info['filename'].append(filename)
                                models_info['accuracy'].append(history.history['acc'][-1])
                                with open('model_info.json', 'w') as file:
                                    json.dump(models_info, file, indent=4)
                                with open('stop_index.txt', 'w') as file:
                                    file.write(str(counter))
                            clear_output(wait=True)
                            counter += 1
    return 

iterate_combinations()

18/17634240 - 0.0% - 156.2
Epoch 1/2
844/844 [==============================] - 3s 3ms/step - loss: 1.5013 - acc: 0.5722 - val_loss: 0.5594 - val_acc: 0.8862
Epoch 2/2
530/844 [=================>............] - ETA: 0s - loss: 0.4820 - acc: 0.8767

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7f8770503e50>
Traceback (most recent call last):
  File "/Users/tannergiddings/opt/anaconda3/lib/python3.9/weakref.py", line 370, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [28]:
with open('stop_index.txt', 'r') as file:
    stop_index = int(file.read())

ValueError: invalid literal for int() with base 10: ''